In [1]:
from lifetimes import generate_data
from lifetimes import datasets
import lifetimes
import numpy as np
import pandas as pd

# Compare Against BFG

In [2]:
bfg = lifetimes.BetaGeoFitter()
cdnow_customers = datasets.load_cdnow_summary()
bfg.fit(cdnow_customers['frequency'], cdnow_customers['recency'], cdnow_customers['T'], iterative_fitting=3)
expected = np.array([0.243, 4.414, 0.793, 2.426])


In [5]:
cdnow_customers.shape

(2357, 4)

In [6]:
# bfg.generate_new_data(size=2400)

In [7]:
bbtf = lifetimes.BetaGeoBetaBinomFitter()

In [8]:
cdnow_df = datasets.load_donations()
donations = cdnow_df

In [9]:
bbtf.fit(
    donations['frequency'],
    donations['recency'],
    donations['n'],
    donations['n_custs'],
)

<lifetimes.BetaGeoBetaBinomFitter: fitted with 22 subjects, alpha: 1.20, beta: 0.75, delta: 2.78, gamma: 0.66>

In [10]:
orig_params = bbtf.params_

In [11]:
size = cdnow_df['n_custs'].sum()

In [12]:
size

11104

In [13]:
# my_data = generate_data.beta_geometric_beta_binom_model(6, 1.204, 0.750, 0.657, 2.783, size=size)
my_data_dfs = []
for _ in range(100):
    my_data_dfs.append(bbtf.generate_new_data(size=size))

In [14]:
my_data = pd.concat(my_data_dfs)

In [15]:
cdnow_df.groupby('frequency')['n_custs'].sum()

frequency
0    3464
1    1823
2    1430
3    1085
4    1036
5    1063
6    1203
Name: n_custs, dtype: int64

In [16]:
my_data['frequency'].value_counts().sort_index()

0.0    344212
1.0    188727
2.0    135461
3.0    111185
4.0    102519
5.0    103193
6.0    125103
Name: frequency, dtype: int64

In [17]:
cdnow_df.groupby('recency')['n_custs'].sum()

recency
0    3464
1    1091
2     890
3     706
4     654
5    1136
6    3163
Name: n_custs, dtype: int64

In [18]:
my_data['recency'].value_counts().sort_index()

0.0    344212
1.0    112378
2.0     85775
3.0     73306
4.0     74695
5.0    106327
6.0    313707
Name: recency, dtype: int64

In [19]:
my_data.head()

,frequency,recency,n,p,theta,alive,customer_id
0,5.0,5.0,6.0,0.608220,0.031538,1.0,0.0
1,0.0,0.0,6.0,0.585566,0.138096,0.0,1.0
2,2.0,2.0,6.0,0.877534,0.286965,0.0,2.0
3,2.0,5.0,6.0,0.375916,0.202427,1.0,3.0
4,0.0,0.0,6.0,0.413106,0.252659,0.0,4.0


In [20]:
grouped_data = my_data.groupby(['frequency', 'recency', 'n'])['customer_id'].count()

In [21]:
grouped_data = grouped_data.reset_index().rename(columns={'customer_id': 'n_custs'})

In [22]:
new_bbtf = lifetimes.BetaGeoBetaBinomFitter()
new_bbtf.fit(
    grouped_data['frequency'],
    grouped_data['recency'],
    grouped_data['n'],
    grouped_data['n_custs'],
)

/Users/zane/Documents/GitHub/lifetimes/lifetimes/fitters/beta_geo_beta_binom_fitter.py:72: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  j = J[:recency_T + 1]


<lifetimes.BetaGeoBetaBinomFitter: fitted with 22 subjects, alpha: 1.21, beta: 0.75, delta: 2.82, gamma: 0.66>

In [23]:
new_params = new_bbtf.params_

In [24]:
orig_params

OrderedDict([('alpha', 1.2035171069321988),
             ('beta', 0.74971518993740249),
             ('gamma', 0.65672205230777436),
             ('delta', 2.7834623711862232)])

In [25]:
new_params

OrderedDict([('alpha', 1.2128584811039143),
             ('beta', 0.75395029743387876),
             ('gamma', 0.66152986164867633),
             ('delta', 2.8174121041002902)])

In [26]:
bbtf.data

,frequency,recency,n,n_custs
0,0,0,6,3464
1,1,1,6,1091
2,1,2,6,277
3,1,3,6,129
4,1,4,6,78
5,1,5,6,119
6,1,6,6,129
7,2,2,6,613
8,2,3,6,255
9,2,4,6,155


In [27]:
bbtf.conditional_probability_alive(1)

0     0.108134
1     0.069462
2     0.254710
3     0.481355
4     0.680383
5     0.828143
6     0.930433
7     0.066415
8     0.298903
9     0.589657
10    0.804555
11    0.930433
12    0.095006
13    0.439593
14    0.766825
15    0.930433
16    0.200178
17    0.696799
18    0.930433
19    0.521974
20    0.930433
21    0.930433
dtype: float64

In [28]:
exp_trans = bbtf.expected_number_of_transactions_in_first_n_periods(6)

In [29]:
exp_trans / exp_trans['model'].sum()

,model
frequency,
0,0.311139
1,0.170088
2,0.121477
3,0.100268
4,0.091672
5,0.092504
6,0.112852


In [30]:
my_data['frequency'].value_counts().sort_index() / my_data.shape[0]

0.0    0.309989
1.0    0.169963
2.0    0.121993
3.0    0.100131
4.0    0.092326
5.0    0.092933
6.0    0.112665
Name: frequency, dtype: float64

In [31]:
my_data.groupby(['frequency', 'recency'])['alive'].mean()

frequency  recency
0.0        0.0        0.107890
1.0        1.0        0.068786
           2.0        0.257759
           3.0        0.488059
           4.0        0.679522
           5.0        0.822720
           6.0        0.934935
2.0        2.0        0.065879
           3.0        0.296113
           4.0        0.586551
           5.0        0.801648
           6.0        0.928077
3.0        3.0        0.092344
           4.0        0.444429
           5.0        0.767564
           6.0        0.930930
4.0        4.0        0.199866
           5.0        0.699042
           6.0        0.928930
5.0        5.0        0.520064
           6.0        0.932407
6.0        6.0        0.931105
Name: alive, dtype: float64

In [32]:
print(self.results.loc[(self.results['frequency'] == 0) & (self.results['recency'] == 0),
                       'alive'].mean())
print(self.results.loc[(self.results['frequency'] == 2) & (self.results['recency'] == 4),
                       'alive'].mean())
print(self.results.loc[(self.results['frequency'] == 6) & (self.results['recency'] == 6),
                       'alive'].mean())
expected_prob_alives = np.array([0.11, 0.59, 0.93])

NameError: name 'self' is not defined